## Paquetes

In [1]:
import pandas as pd
import numpy as np
import re
import requests as req
from datetime import date

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import sys
from src.scraping_functions import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
opciones.add_experimental_option("detach", True)
path = "/mnt/d/Google Drive/Iron hack/Instalaciones/chromedriver.exe"

## Importar  y limpiar csv

In [4]:
data = pd.read_csv("data/residential-stay-in-covid19.csv",encoding = "ISO-8859-1")

In [5]:
data.shape

(91933, 3)

In [6]:
data.sample(5)

,Entity,Day,Increase in Residential Stay
28025,France,01-11-2020,11.000
76501,South Korea,19-11-2020,0.714
76774,South Korea,19-08-2021,5.857
70189,Romania,27-01-2021,5.857
86536,United Kingdom,28-12-2020,19.000


In [7]:
data.isna().sum()

Entity                          0
Day                             0
Increase in Residential Stay    0
dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91933 entries, 0 to 91932
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Entity                        91933 non-null  object 
 1   Day                           91933 non-null  object 
 2   Increase in Residential Stay  91933 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [9]:
nuevas_columnas = {"Entity": "country", "Day":"date"}
data.rename(columns=nuevas_columnas, inplace=True)

In [10]:
data["date"]=pd.to_datetime(data["date"],dayfirst=True)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91933 entries, 0 to 91932
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   country                       91933 non-null  object        
 1   date                          91933 non-null  datetime64[ns]
 2   Increase in Residential Stay  91933 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.1+ MB


In [12]:
eeuu=data[data.country=="United States"]

In [13]:
eeuu.date.min()

Timestamp('2020-02-17 00:00:00')

In [14]:
eeuu.drop(columns = ["country"],axis=1,inplace= True)

/tmp/ipykernel_2441/2038731044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeuu.drop(columns = ["country"],axis=1,inplace= True)


In [15]:
eeuu.head(5)

,date,Increase in Residential Stay
86935,2020-02-17,1.000
86936,2020-02-18,1.000
86937,2020-02-19,0.800
86938,2020-02-20,0.833
86939,2020-02-21,0.714


## Scrapeo

In [16]:
dict_tablas={}
dict_df={}
for e in ["NFLX","DIS","AMC"]:
    dict_tablas[e] = yahoo_finance(e, 771, 3, 1, "history")    
    dict_df[e]=pd.DataFrame([i.split(" ") for i in dict_tablas[e].split("\n")]
                            ,columns=["month","day","year", f"open_{e}", f"high_{e}", f"low_{e}", f"close_{e}", f"adj_close_{e}", f"volume_{e}"])

In [17]:
for e in list(dict_df.keys()):
    dict_df[e]['date'] = dict_df[e]["year"]+" "+dict_df[e]["month"]+" "+dict_df[e]["day"]
    dict_df[e]['date'] = dict_df[e]['date'].apply(lambda x : x.rstrip(","))
    dict_df[e]['date'] = pd.to_datetime(dict_df[e]['date'], format='%Y %b %d')
    dict_df[e].drop(columns = ["year","month","day",f"open_{e}", f"high_{e}", f"low_{e}", f"adj_close_{e}", f"volume_{e}" ]
                    ,axis=1,inplace= True)
    dict_df[e] = dict_df[e].iloc[:,[1,0]]


In [18]:
dict_df["NFLX"].sample(5)

,date,close_NFLX
156,2021-06-24,518.06
45,2021-12-01,617.77
263,2021-01-21,579.84
465,2020-04-02,370.08
70,2021-10-26,668.52


In [19]:
tabla = eeuu.merge(dict_df["NFLX"], left_on='date', right_on='date').merge(dict_df["DIS"], left_on='date', right_on='date').merge(dict_df["AMC"], left_on='date', right_on='date')

In [20]:
tabla.to_csv("data/eeuu.csv",index=False)

In [21]:
tabla.sample(5)

,date,Increase in Residential Stay,close_NFLX,close_DIS,close_AMC
348,2021-07-02,4.000,533.98,177.11,51.96
319,2021-05-21,4.714,497.89,172.40,12.08
18,2020-03-12,-0.143,315.25,91.81,2.91
222,2020-12-31,14.286,540.73,181.18,2.12
437,2021-11-08,4.429,651.45,176.87,45.06


## Api

In [2]:
start_date=date(2022, 2, 3)
end_date=date(2022, 2, 5)

daterange = pd.date_range(start_date, end_date)
datos_tiempo=[]

for single_date in daterange:
    url=f'http://api.weatherapi.com/v1/history.json?key=40e98f4fcb06417bbe1182003220702&q=New-York&dt={single_date}'
    datos_tiempo.append((req.get(url).json()["forecast"]["forecastday"][0]["date"],req.get(url).json()["forecast"]["forecastday"][0]["day"]["avgtemp_c"]))
datos_tiempo

[('2022-02-03', 1.9), ('2022-02-04', -3.1), ('2022-02-05', -6.9)]